### 경고 무시 

In [1]:
import warnings

warnings.filterwarnings(action='ignore') 

### 필요 라이브러리 Import

In [2]:
import tensorflow as tf 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf 

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model 
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import TensorBoard

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

### 함수 정의

In [3]:
def load_data():
    (train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()
    train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=10)
    
    train_X = train_X.reshape(-1, 784)
    train_X = train_X.astype('float32')
    train_X /= 255
    
    val_X = val_X.reshape(-1, 784)
    val_X = val_X.astype('float32')
    val_X /= 255
    
    test_X = test_X.reshape(-1, 784)
    test_X = test_X.astype('float32')
    test_X /= 255
    
    train_y = to_categorical(train_y)
    val_y = to_categorical(val_y)
    test_y = to_categorical(test_y)
    
    data = dict()
    data['train_X'] = train_X
    data['train_y'] = train_y
    data['val_X'] = val_X
    data['val_y'] = val_y
    data['test_X'] = test_X
    data['test_y'] = test_y
    
    return data 

In [4]:
data = load_data()

### Callback 생성 

In [7]:
class RocAUCScore(Callback):
    def __init__(self, training_data, validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        super(RocAUCScore, self).__init__()
        
    # 각 epoch 말에 ROC AUC 점수 계산 
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\n *** ROC AUC Score: %s - roc-auc_val: %s ***' % (str(roc), str(roc_val)))

        return 

In [8]:
def create_callbacks(data):
    tensorboard_callback = TensorBoard(log_dir='./ch4_tb_log/mlp', histogram_freq=1, batch_size=32, 
                                        write_graph=True, write_grads=False)
    roc_auc_callback = RocAUCScore(
        training_data=(data['train_X'], data['train_y']),
        validation_data=(data['val_X'], data['val_y']))
    checkpoint_callback = ModelCheckpoint(
        filepath='./model-weights.{epoch:02d}-{val_acc:.6f}.hdf5',
        monitor='val_acc', verbose=1, save_best_only=True)

    return [tensorboard_callback, roc_auc_callback, checkpoint_callback]

In [6]:
def build_network(input_features=None):
    inputs = Input(shape=(input_features, ), name='input')
    x = Dense(512, activation='relu', name='hidden1',kernel_regularizer='l2')(inputs)
    x = Dense(256, activation='relu', name='hidden2',kernel_regularizer='l2')(x)
    x = Dense(128, activation='relu', name='hidden3',kernel_regularizer='l2')(x)
    prediction = Dense(10, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
callbacks = create_callbacks(data)

### 모델 학습 

In [10]:
model = build_network(data['train_X'].shape[1])
model.fit(x=data['train_X'], y=data['train_y'], 
          batch_size=30, 
          epochs=50, 
          validation_data=(data['val_X'], data['val_y']),
         verbose=1, callbacks=callbacks)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 48000 samples, validate on 12000 samples
Epoch 1/50
47850/48000 [============================>.] - ETA: 0s - loss: 1.2013 - acc: 0.8828
 *** ROC AUC Score: 0.9949597858814488 - roc-auc_val: 0.9947251370669369 ***

Epoch 00001: val_acc improved from -inf to 0.91508, saving model to ./model-weights.01-0.915083.hdf5
48000/48000 [==============================] - 28s 573us/sample - loss: 1.1996 - acc: 0.8829 - val_loss: 0.6580 - val_acc: 0.9151
Epoch 2/50
47700/48000 [============================>.] - ETA: 0s - loss: 0.6056 - acc: 0.9183
 *** ROC AUC Score: 0.9962808116932111 - roc-auc_val: 0.9960371192639192 ***

Epoch 00002: val_acc improved from 0.91508 to 0.92592, saving model to ./model-weights.02-0.925917.hdf5
48000/48000 [==============================] - 29s 598us/sample - loss: 0.6056 - acc: 0.9182 - val_loss: 0.5663 - val_acc: 0.9259
Epoch 3/50
47790/4800

Epoch 22/50
47700/48000 [============================>.] - ETA: 0s - loss: 0.3338 - acc: 0.9513
 *** ROC AUC Score: 0.9984430514541763 - roc-auc_val: 0.9980471334611234 ***

Epoch 00022: val_acc did not improve from 0.95525
48000/48000 [==============================] - 17s 361us/sample - loss: 0.3338 - acc: 0.9513 - val_loss: 0.3310 - val_acc: 0.9528
Epoch 23/50
47760/48000 [============================>.] - ETA: 0s - loss: 0.3307 - acc: 0.9521
 *** ROC AUC Score: 0.9983264927430808 - roc-auc_val: 0.9979736336163059 ***

Epoch 00023: val_acc did not improve from 0.95525
48000/48000 [==============================] - 17s 360us/sample - loss: 0.3306 - acc: 0.9521 - val_loss: 0.3383 - val_acc: 0.9467
Epoch 24/50
47730/48000 [============================>.] - ETA: 0s - loss: 0.3298 - acc: 0.9512
 *** ROC AUC Score: 0.9987080852509399 - roc-auc_val: 0.9983052795808357 ***

Epoch 00024: val_acc did not improve from 0.95525
48000/48000 [==============================] - 17s 364us/sample - lo

Epoch 45/50
47670/48000 [============================>.] - ETA: 0s - loss: 0.3079 - acc: 0.9537
 *** ROC AUC Score: 0.9982446589963752 - roc-auc_val: 0.9977878091541953 ***

Epoch 00045: val_acc did not improve from 0.95767
48000/48000 [==============================] - 18s 366us/sample - loss: 0.3083 - acc: 0.9534 - val_loss: 0.3348 - val_acc: 0.9442
Epoch 46/50
47790/48000 [============================>.] - ETA: 0s - loss: 0.3071 - acc: 0.9534
 *** ROC AUC Score: 0.9984710628422311 - roc-auc_val: 0.9980803317614896 ***

Epoch 00046: val_acc did not improve from 0.95767
48000/48000 [==============================] - 18s 372us/sample - loss: 0.3071 - acc: 0.9534 - val_loss: 0.3256 - val_acc: 0.9477
Epoch 47/50
47880/48000 [============================>.] - ETA: 0s - loss: 0.3122 - acc: 0.9509
 *** ROC AUC Score: 0.9985079120048466 - roc-auc_val: 0.9981645776399724 ***

Epoch 00047: val_acc did not improve from 0.95767
48000/48000 [==============================] - 18s 372us/sample - lo

### 모델 평가

In [11]:
y_softmax = model.predict(data['test_X'])
y_hat = y_softmax.argmax(axis=-1)

from sklearn.metrics import classification_report 
import numpy as np 

print(classification_report(np.argmax(data['test_y'], axis=-1), y_hat))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       980
           1       0.94      1.00      0.97      1135
           2       0.94      0.97      0.95      1032
           3       0.91      0.96      0.94      1010
           4       0.93      0.97      0.95       982
           5       0.96      0.92      0.94       892
           6       0.96      0.95      0.96       958
           7       0.97      0.94      0.96      1028
           8       0.93      0.94      0.94       974
           9       0.97      0.89      0.93      1009

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

